In [ ]:
# ==============================
# 🔹 Safe NumPy & Gradio Setup for Colab
# ==============================

# 1. Downgrade NumPy to match Colab's preinstalled packages
!pip install --quiet --force-reinstall numpy==1.26.4

# 2. Install Gradio compatible with NumPy 1.x
!pip install --quiet gradio==4.44.0

# 3. Restart runtime to apply changes
import os
os.kill(os.getpid(), 9)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.0.3 which is incompatible.
arviz 0.22.0 requires pandas>=2.1.0, but you have pandas 2.0.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 2.0.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
mizani 0.13.5 requires pandas>=2.2.0, but you have pandas 2.0.3 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
xarray 2025.7.1 requires pandas>=2.2, but you have pandas 2.0.3 which is incompatible.
opencv-python-headless 4.12.0.8

In [ ]:
!pip install --quiet --upgrade pip
!pip install --quiet gradio==4.44.0
!pip install --quiet crewai==0.41.1
!pip install --quiet langchain-google-genai==1.0.10
!pip install --quiet numpy==1.24.3
!pip install --quiet pandas==2.0.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio-client 1.3.0 requires websockets<13.0,>=10.0, but you have websockets 15.0.1 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.36.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 whic

In [ ]:
import os
import json
import time
from datetime import datetime
from typing import Dict, List, Any

# Set environment variables for better compatibility
os.environ['GRPC_PYTHON_BUILD_SYSTEM_OPENSSL'] = '1'
os.environ['GRPC_PYTHON_BUILD_SYSTEM_ZLIB'] = '1'

try:
    import gradio as gr
    print("✅ Gradio imported successfully")
except ImportError as e:
    print(f"❌ Gradio import error: {e}")
    print("Run: !pip install gradio==4.44.0")

try:
    from crewai import Agent, Task, Crew, Process
    print("✅ CrewAI imported successfully")
except ImportError as e:
    print(f"❌ CrewAI import error: {e}")
    print("Run: !pip install crewai==0.41.1")

try:
    from langchain_google_genai import ChatGoogleGenerativeAI
    print("✅ LangChain Google GenAI imported successfully")
except ImportError as e:
    print(f"❌ LangChain Google GenAI import error: {e}")
    print("Run: !pip install langchain-google-genai==1.0.10")



✅ Gradio imported successfully


/usr/local/lib/python3.11/dist-packages/google/colab/html/_background_server.py:103: DeprecationWarning: make_current is deprecated; start the event loop first
  ioloop.make_current()
/usr/local/lib/python3.11/dist-packages/crewai/telemetry/telemetry.py:9: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as sp

✅ CrewAI imported successfully
✅ LangChain Google GenAI imported successfully


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

def setup_llm():
    """Initialize the Gemini LLM"""
    try:
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            temperature=0.7,
            google_api_key=GOOGLE_API_KEY
        )
        print("✅ Gemini LLM initialized successfully")
        return llm
    except Exception as e:
        print(f"❌ LLM setup error: {e}")
        return None


In [ ]:

class StartupAgents:
    def __init__(self, llm):
        self.llm = llm

    def create_problem_validator(self):
        """Agent 1: Problem Validator"""
        return Agent(
            role='Problem Validator',
            goal='Analyze startup ideas to determine if they solve real-world problems',
            backstory="""You are an experienced product manager and market analyst with 15 years
            of experience in identifying genuine market needs. You have a keen eye for distinguishing
            between real problems that people will pay to solve versus nice-to-have features.
            You've seen countless startups fail because they built solutions to problems that
            didn't exist or weren't painful enough.""",
            verbose=True,
            allow_delegation=False,
            llm=self.llm
        )

    def create_market_researcher(self):
        """Agent 2: Market Researcher"""
        return Agent(
            role='Market Researcher',
            goal='Research market size, target audience, and competitive landscape',
            backstory="""You are a senior market research analyst with expertise in TAM/SAM/SOM
            analysis, customer segmentation, and competitive intelligence. You have access to
            industry reports, market trends, and consumer behavior data. You excel at identifying
            market opportunities and understanding customer demographics, psychographics, and
            buying behaviors.""",
            verbose=True,
            allow_delegation=False,
            llm=self.llm
        )

    def create_business_model_builder(self):
        """Agent 3: Business Model Builder"""
        return Agent(
            role='Business Model Strategist',
            goal='Design viable business models and monetization strategies',
            backstory="""You are a business strategy consultant with MBA-level expertise in
            business model innovation. You've helped dozens of startups find product-market fit
            and scale their revenue. You understand various monetization strategies including
            SaaS, marketplace, freemium, subscription, and transaction-based models. You think
            critically about unit economics, customer acquisition costs, and lifetime value.""",
            verbose=True,
            allow_delegation=False,
            llm=self.llm
        )

    def create_risk_analyzer(self):
        """Agent 4: Risk Analyzer"""
        return Agent(
            role='Risk Analyzer',
            goal='Identify potential risks and develop mitigation strategies',
            backstory="""You are a venture capital partner and former startup founder who has
            seen both spectacular successes and devastating failures. You have a talent for
            identifying potential risks before they become fatal problems. You understand
            technical risks, market risks, competitive risks, regulatory risks, and operational
            risks. You provide practical, actionable mitigation strategies.""",
            verbose=True,
            allow_delegation=False,
            llm=self.llm
        )


In [ ]:

class StartupTasks:
    def __init__(self):
        pass

    def create_problem_validation_task(self, startup_idea, agent):
        """Task for Problem Validator"""
        return Task(
            description=f"""
            Analyze this startup idea: "{startup_idea}"

            Provide a comprehensive problem validation analysis including:
            1. Problem Definition: Clearly articulate the problem being solved
            2. Problem Severity: Rate the pain level (1-10) and explain why
            3. Target User Analysis: Who specifically faces this problem?
            4. Current Solutions: How do people solve this problem today?
            5. Problem Frequency: How often do users encounter this problem?
            6. Willingness to Pay: Evidence that users would pay for a solution
            7. Problem Validation Score: Overall score (1-10) with justification

            Be critical and honest. If the problem seems weak or fake, say so.
            """,
            agent=agent,
            expected_output="A detailed problem validation report in JSON format with scores and analysis"
        )

    def create_market_research_task(self, startup_idea, agent):
        """Task for Market Researcher"""
        return Task(
            description=f"""
            Research the market for this startup idea: "{startup_idea}"

            Provide comprehensive market analysis including:
            1. Market Size (TAM/SAM/SOM estimates)
            2. Target Customer Segments (demographics, psychographics)
            3. Market Trends (growing/declining, key drivers)
            4. Competitive Analysis (direct and indirect competitors)
            5. Market Entry Barriers
            6. Customer Acquisition Channels
            7. Market Attractiveness Score (1-10) with reasoning

            Base your analysis on realistic market assumptions and industry knowledge.
            """,
            agent=agent,
            expected_output="A structured market research report with data-driven insights"
        )

    def create_business_model_task(self, startup_idea, agent):
        """Task for Business Model Builder"""
        return Task(
            description=f"""
            Design a viable business model for: "{startup_idea}"

            Create a comprehensive business model including:
            1. Revenue Streams (primary and secondary)
            2. Pricing Strategy (model and justification)
            3. Customer Acquisition Strategy
            4. Key Resources and Partnerships
            5. Cost Structure Analysis
            6. Unit Economics (CAC, LTV, gross margins)
            7. Growth Strategy and Scalability
            8. Business Model Viability Score (1-10)

            Focus on practical, implementable strategies with realistic assumptions.
            """,
            agent=agent,
            expected_output="A detailed business model canvas with financial projections"
        )

    def create_risk_analysis_task(self, startup_idea, agent):
        """Task for Risk Analyzer"""
        return Task(
            description=f"""
            Identify and analyze risks for: "{startup_idea}"

            Provide comprehensive risk assessment including:
            1. Technical Risks (development, scalability, security)
            2. Market Risks (demand, competition, timing)
            3. Financial Risks (funding, cash flow, unit economics)
            4. Operational Risks (team, execution, partnerships)
            5. Regulatory/Legal Risks (compliance, IP, privacy)
            6. Strategic Risks (positioning, differentiation)
            7. Risk Mitigation Strategies for each identified risk
            8. Overall Risk Level (Low/Medium/High) with explanation

            Be thorough and consider both obvious and hidden risks.
            """,
            agent=agent,
            expected_output="A comprehensive risk assessment with mitigation strategies"
        )

# ============================================================================
# SECTION 6: CREW ORCHESTRATION
# Main crew that coordinates all agents
# ============================================================================

class StartupEvaluatorCrew:
    def __init__(self, llm):
        self.llm = llm
        self.agents = StartupAgents(llm)
        self.tasks = StartupTasks()

    def create_crew(self, startup_idea):
        """Create and configure the crew"""
        # Initialize agents
        problem_validator = self.agents.create_problem_validator()
        market_researcher = self.agents.create_market_researcher()
        business_model_builder = self.agents.create_business_model_builder()
        risk_analyzer = self.agents.create_risk_analyzer()

        # Create tasks
        problem_task = self.tasks.create_problem_validation_task(startup_idea, problem_validator)
        market_task = self.tasks.create_market_research_task(startup_idea, market_researcher)
        business_task = self.tasks.create_business_model_task(startup_idea, business_model_builder)
        risk_task = self.tasks.create_risk_analysis_task(startup_idea, risk_analyzer)

        # Create crew
        crew = Crew(
            agents=[problem_validator, market_researcher, business_model_builder, risk_analyzer],
            tasks=[problem_task, market_task, business_task, risk_task],
            process=Process.sequential,
            verbose=True
        )

        return crew

    def evaluate_startup(self, startup_idea: str) -> Dict[str, Any]:
        """Main evaluation function"""
        try:
            print(f"🚀 Starting evaluation for: {startup_idea}")

            # Create and run crew
            crew = self.create_crew(startup_idea)
            result = crew.kickoff()

            # Parse and format results
            evaluation_results = {
                "startup_idea": startup_idea,
                "evaluation_timestamp": datetime.now().isoformat(),
                "problem_validation": "Analysis completed",
                "market_research": "Research completed",
                "business_model": "Model designed",
                "risk_analysis": "Risks identified",
                "full_report": str(result)
            }

            return evaluation_results

        except Exception as e:
            return {
                "error": f"Evaluation failed: {str(e)}",
                "startup_idea": startup_idea,
                "timestamp": datetime.now().isoformat()
            }


In [ ]:

def create_gradio_interface():
    """Create the Gradio interface"""

    # Initialize the system
    llm = setup_llm()
    if llm is None:
        raise Exception("Failed to initialize LLM. Check your API key.")

    evaluator = StartupEvaluatorCrew(llm)

    def evaluate_idea(startup_idea, description):
        """Process startup idea evaluation"""
        if not startup_idea.strip():
            return "❌ Please enter a startup idea to evaluate."

        # Combine idea and description
        full_idea = f"{startup_idea.strip()}"
        if description.strip():
            full_idea += f" - {description.strip()}"

        try:
            # Show progress
            progress_msg = "🔄 Evaluating your startup idea...\n"
            progress_msg += "👥 Problem Validator is analyzing...\n"

            # Run evaluation
            results = evaluator.evaluate_startup(full_idea)

            if "error" in results:
                return f"❌ Evaluation Error: {results['error']}"

            # Format results for display
            report = f"""
# 📊 STARTUP EVALUATION REPORT

**Startup Idea:** {results['startup_idea']}
**Evaluation Date:** {results['evaluation_timestamp']}

---

## 📋 EVALUATION SUMMARY

✅ **Problem Validation:** {results['problem_validation']}
✅ **Market Research:** {results['market_research']}
✅ **Business Model:** {results['business_model']}
✅ **Risk Analysis:** {results['risk_analysis']}

---

## 📄 DETAILED ANALYSIS

{results['full_report']}

---

*Evaluation completed by AI Multi-Agent System*
            """

            return report

        except Exception as e:
            return f"❌ System Error: {str(e)}\n\nPlease check your API key and try again."

    # Create interface
    with gr.Blocks(title="🚀 Startup Idea Evaluator", theme=gr.themes.Soft()) as interface:

        gr.Markdown("""
        # 🚀 Multi-Agent Startup Idea Evaluator

        Get comprehensive analysis of your startup idea from 4 AI specialists:
        - 🔍 **Problem Validator** - Validates real-world problems
        - 📊 **Market Researcher** - Analyzes market opportunity
        - 💼 **Business Model Builder** - Designs monetization strategy
        - ⚠️ **Risk Analyzer** - Identifies potential risks
        """)

        with gr.Row():
            with gr.Column(scale=1):
                startup_input = gr.Textbox(
                    label="💡 Startup Idea",
                    placeholder="e.g., AI-powered nutrition coach app",
                    lines=2
                )

                description_input = gr.Textbox(
                    label="📝 Description (Optional)",
                    placeholder="Provide additional details about your idea...",
                    lines=3
                )

                evaluate_btn = gr.Button("🔍 Evaluate Idea", variant="primary", size="lg")

                gr.Markdown("""
                ### 📝 Sample Ideas to Try:
                - AI-powered personal fitness coach
                - Sustainable packaging for e-commerce
                - Virtual reality therapy for anxiety
                - Smart home energy optimization system
                """)

            with gr.Column(scale=2):
                output_report = gr.Textbox(
                    label="📊 Evaluation Report",
                    lines=25,
                    max_lines=50,
                    show_copy_button=True
                )

        evaluate_btn.click(
            fn=evaluate_idea,
            inputs=[startup_input, description_input],
            outputs=output_report,
            show_progress=True
        )

        gr.Markdown("""
        ---
        ### 🔧 System Architecture:
        - **Backend:** Google Gemini Pro API
        - **Orchestration:** CrewAI Multi-Agent Framework
        - **Frontend:** Gradio Interface
        - **Process:** Sequential agent execution with knowledge sharing
        """)

    return interface


In [ ]:

def main():
    """Main execution function"""
    print("🚀 Starting Startup Idea Evaluator System...")

    # Check API key
    if GOOGLE_API_KEY == "YOUR_GOOGLE_API_KEY_HERE":
        print("❌ Please set your Google API key in SECTION 3")
        print("Get your API key from: https://makersuite.google.com/app/apikey")
        return

    try:
        # Create and launch interface
        interface = create_gradio_interface()
        interface.launch(
            share=True,  # Creates public link for sharing
            debug=True,  # Enable debug mode
            server_name="0.0.0.0",  # Allow external connections
            server_port=7860  # Default Gradio port
        )

    except Exception as e:
        print(f"❌ Launch Error: {e}")
        print("\n🔧 Troubleshooting Steps:")
        print("1. Check if all packages are installed correctly")
        print("2. Verify your Google API key is valid")
        print("3. Restart runtime and run sections 1-2 again")

# ============================================================================
# SECTION 9: EXAMPLE TEST CASE
# Test with sample startup idea
# ============================================================================

def test_sample_idea():
    """Test the system with a sample startup idea"""
    print("🧪 Testing with sample idea...")

    llm = setup_llm()
    if llm is None:
        print("❌ Cannot test - LLM initialization failed")
        return

    evaluator = StartupEvaluatorCrew(llm)
    sample_idea = "AI-powered nutrition coach that creates personalized meal plans based on user's health goals, dietary restrictions, and food preferences"

    print(f"Testing idea: {sample_idea}")
    results = evaluator.evaluate_startup(sample_idea)

    print("✅ Test Results:")
    print(json.dumps(results, indent=2))

# Run the system
if __name__ == "__main__":
    # Uncomment to test with sample idea first
    # test_sample_idea()

    # Launch main application
    main()

/usr/local/lib/python3.11/dist-packages/gradio/routes.py:1215: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
/usr/local/lib/python3.11/dist-packages/fastapi/applications.py:4495: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  return self.router.on_event(event_type)


🚀 Starting Startup Idea Evaluator System...
✅ Gemini LLM initialized successfully
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().


/usr/local/lib/python3.11/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://107e3974acc9a64ac5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/starlette/templating.py:162: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(


...

I now can give a great answer
Final Answer:
```json
{
  "startup_idea": "AI-powered plant health scanner that allows farmers or gardeners to take a photo of their plant using a mobile app, instantly detect diseases, nutrient deficiencies, or pest infestations, and receive actionable treatment suggestions along with preventive care tips. The system could also recommend nearby agricultural shops or online suppliers for necessary products.",
  "problem_validation_analysis": {
    "problem_definition": "The core problem is the significant difficulty and time-consuming nature for both commercial farmers and hobby gardeners to accurately and quickly diagnose plant health issues (such as diseases, nutrient deficiencies, or pest infestations). This often leads to delayed or incorrect interventions, resulting in crop loss, reduced yields, wasted resources (time, money on ineffective treatments), and ultimately, financial losses for farmers or emotional frustration and replacement costs for gard

Thought: I now can give a great answer
Final Answer:### Market Analysis: AI-Powered Plant Health Scanner

**Startup Idea:** "AI-powered plant health scanner that allows farmers or gardeners to take a photo of their plant using a mobile app, instantly detect diseases, nutrient deficiencies, or pest infestations, and receive actionable treatment suggestions along with preventive care tips. The system could also recommend nearby agricultural shops or online suppliers for necessary products."

---

#### 1. Market Size (TAM/SAM/SOM Estimates)

The market for an AI-powered plant health scanner spans two major sectors: commercial agriculture (especially small to medium-scale farms) and the vast consumer gardening/horticulture market.

*   **Total Addressable Market (TAM):** The total revenue opportunity if every potential customer globally adopted this solution.
    *   **Commercial Agriculture (Global):** The global smart agriculture market, which includes precision farming, agricultural IoT

/usr/local/lib/python3.11/dist-packages/starlette/templating.py:162: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(


Thought: I now can give a great answer
Final Answer:
## Business Model Design: AI-Powered Plant Health Scanner

This comprehensive business model is designed for the "AI-powered plant health scanner" startup, focusing on practical, implementable strategies and realistic assumptions, leveraging insights from the provided market analysis.

---

### 1. Revenue Streams

The business model will employ a multi-faceted approach to revenue generation, catering to both B2C (hobby gardeners) and B2B/B2C hybrid (small-to-medium commercial farmers) segments.

*   **Primary Revenue Stream: Subscription Model (Tiered)**
    *   **Freemium Tier (Base Product):** Free access to basic plant identification and limited disease/deficiency diagnosis (e.g., 3 scans per month). This serves as a lead magnet and allows users to experience the core value proposition, lowering the barrier to entry and facilitating virality.
    *   **"Gardener Premium" Subscription:** Targeted at serious hobby gardeners and home

/usr/local/lib/python3.11/dist-packages/starlette/templating.py:162: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(
